In [1]:
from difflib import SequenceMatcher
import json
import pickle
import pandas as pd
from tqdm import tqdm

indexPath = "E:\Technologie\Master\Webm\subtitles_all.pkl"
movies_json = "data/movies.json"

output_file = "E:\Technologie\Master\Webm\subtitles_all_linked.pkl"

SLOW = False

In [2]:
# load the dataframe from the pickle file
df = pd.read_pickle(indexPath)

In [3]:
# load the json file
with open(movies_json,encoding="utf8") as json_file:
    data = json.load(json_file)

In [4]:
# reduce the sizes for test purposes
movies = data[:]
df_sub_index = df[:]

In [5]:
# delete the rows when there is no "title" key in the data json file
for i in range(len(movies)):
    if "uuid" not in movies[i] or 'title' not in movies[i] or "year" not in movies[i]:
        movies[i] = None

# delete the empty rows
movies = list(filter(None, movies))

In [6]:
df_movies = pd.DataFrame(movies)

In [7]:
# keep only columns we need
df_movies = df_movies[["uuid",'title', 'year']]

In [8]:
# see the data structures
print(df_movies.columns)
print(df_sub_index.columns)

Index(['uuid', 'title', 'year'], dtype='object')
Index(['IDSubtitle', 'MovieName', 'MovieYear'], dtype='object')


### Rapid method

In [9]:
# Create a temporary column in both dataframes for merging
df_movies['key'] = df_movies['title'].str.lower() + df_movies['year'].astype(str)
df_sub_index['key'] = df_sub_index['MovieName'].str.lower() + df_sub_index['MovieYear'].astype(str)

# Merge the two dataframes on the 'key' column
df_merged = pd.merge(df_sub_index, df_movies, on='key', how='left')

# Drop the temporary 'key' columns
df_movies.drop(columns=['key'], inplace=True)
df_sub_index.drop(columns=['key'], inplace=True)
df_merged.drop(columns=['key'], inplace=True)

C:\Users\massi2\AppData\Local\Temp\ipykernel_26356\3423333825.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_index['key'] = df_sub_index['MovieName'].str.lower() + df_sub_index['MovieYear'].astype(str)


### Slow method

In [10]:
if SLOW == True:

    # function that matches an int only to another int if they are within a certain range
    def matchYear(year,cYear, yearDelta=1):
        if abs(year - cYear) > yearDelta:
            return False
        else:
            return True

    # function that deternmines the similarity between two movies titles and years
    def match_movies(title, year, title2, year2, threshold=0.95,yearDelta=1):
        if SequenceMatcher(None, title, title2).ratio() > threshold and matchYear(int(year),int(year2),yearDelta):
            return True
        else:
            return False

    # add a column to the dataframe that contains the movie uuid
    df_sub_index['movie_uuid'] = ""

    matches = []
    length = len(df_movies)

    for i,movie in tqdm(df_movies.iterrows(),total=length):
        for index, row in df_sub_index.iterrows():
            if match_movies(movie["title"],movie["year"],row["MovieName"],row["MovieYear"],threshold=0.9,yearDelta=1):
                df_sub_index.loc[index,'movie_uuid'] = movie["uuid"]
                break

### Results

In [11]:
#print the rows that have a movie uuid
# df_sub_index[df_sub_index['movie_uuid'] != ""]

In [12]:


# print the rows there movie uuid is not NaN
df_out = df_merged[df_merged['uuid'].notna()].drop(columns=["title","year"])
df_out

,IDSubtitle,MovieName,MovieYear,uuid
8,24,Ghost Ship,2002,b668a638f30111edb451b8aeed79c0cc
10,28,I Spy,2002,9adb338ef30311edb451b8aeed79c0cc
11,29,Dead Ringers,1988,3a81ffb0f30111edb451b8aeed79c0cc
21,88,Hi no tori,1978,33fc6720f31a11edb451b8aeed79c0cc
23,98,The Return,2003,649d1528f30111edb451b8aeed79c0cc
...,...,...,...,...
96604,9177680,Anything's Possible,2022,f8576998f31b11edb451b8aeed79c0cc
96616,9178231,Kill or Cure,1962,267cb10ef30111edb451b8aeed79c0cc
96623,9179258,Born Reckless,1958,e0fa63a0f30b11edb451b8aeed79c0cc
96631,9179688,Shut In,2022,c814b906f31c11edb451b8aeed79c0cc


In [13]:
# save the dataframe to a pickle file
df_out.to_pickle(output_file)